# Calculation of new load profile: base load + EV load

In [ ]:
#import notebook
import nbimporter
import load_allocation
import EV_profile as ev

# Functions

In [ ]:
def sum_of_base_load_and_ev_load(request,method="aggr"):
    """
    Sum base load optimised for each building based on the PV size and ev load for each building
    request: the dict-like request object passed by Flask
    method: String with value 1) aggr: Optimisation by aggregated load 2) roofeff: optimisation by roof efficiency
    Returns: sum of aggregated loads
    
    """
    
    
    if method == "aggr":
        optimal_size,base_load,building_type,no_of_charge_points = load_allocation.pv_aggregated_load_profile_optimiser(request)
    else:
        optimal_size,base_load,building_type,no_of_charge_points = load_allocation.pv_allocate_size_by_roof_eff_optimiser(request)
    aggr_load_kw = []
    for load, charge_points in zip(base_load, no_of_charge_points):
        start_date = load.sort_index(ascending=True).index[0].date()
        ev_load = ev.EV_profile(building_type,0.2,start_date,charge_points)
        tmp_aggr_load = load.add(ev_load,fill_value=0)
        aggr_load_kw.append(tmp_aggr_load)
    return(aggr_load_kw)    